## One time

In [ ]:
%pip install google_spreadsheet
%pip install google-auth-oauthlib
%pip install pandas

## Download data

Windows ghseet id: GSHEET_ID

Mac gheest id: GSHEET_ID

In [81]:
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']


def download(spreadhseetId, rangeName):
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_269207949965-otm96920haoqo8hnmp97c7d8atjm2hfg.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=spreadhseetId,
                                range=rangeName).execute()
    values_input = result_input.get('values', [])

    if not values_input:
        print('No data found.')
    return pd.DataFrame(values_input[1:], columns=values_input[0])

In [ ]:
mac_df = download("DOC_ID", "ROWS_AND_COLUMNS")
mac_df.head()

In [44]:
mac_data = {}
for sheet in ["PrivacyTests", "BrowserLeaks", "Privacy Test Pages (DDG)"]:
    mac_data[sheet] = download("DOC_ID", sheet + "ROWS_AND_COLUMNS")

In [45]:
win_data = {}
for sheet in ["PrivacyTests", "BrowserLeaks", "Privacy Test Pages (DDG)"]:
    win_data[sheet] = download("DOC_ID", sheet + "ROWS_AND_COLUMNS")

## Unification

In [ ]:
# Initialize the mega dataframe
mega_df = pd.DataFrame(columns=["sheet", "test", "os", "browser", "result"])

# Iterate through mac_data
for sheet in mac_data:
    for _, row in mac_data[sheet].iterrows():
        # if majority of row is string "N/A", skip
        if row.isin(["N/A"]).sum() > 7:
            continue
        for browser in mac_data[sheet].columns:
            new_row = pd.DataFrame([{
                "sheet": sheet,
                "test": row[0], # assuming 'test' is a column in your mac_data DataFrames
                "os": "mac",
                "browser": browser,
                "result": row[browser]
            }])
            mega_df = pd.concat([mega_df, new_row], ignore_index=True)

# Iterate through win_data
for sheet in win_data:
    for _, row in win_data[sheet].iterrows():
        # if majority of row is N/A, skip
        if row.isin(["N/A"]).sum() > 7:
            continue
        for browser in win_data[sheet].columns:
            new_row = pd.DataFrame([{
                "sheet": sheet,
                "test": row[0], # assuming 'test' is a column in your win_data DataFrames
                "os": "win",
                "browser": browser,
                "result": row[browser]
            }])
            mega_df = pd.concat([mega_df, new_row], ignore_index=True)

# Display the first few rows of mega_df
print(mega_df.head())

In [ ]:
# remove rows with browser not in Brave, Chrome, Chromium, DuckDuckGo, Edge, Epicm Libre Wolf, Opera, Firefox, Safari, Tor, Vivaldi, or Waterfox

mega_df = mega_df[mega_df['browser'].isin(['Brave', 'Chrome', 'Chromium', 'DuckDuckGo', 'Edge', 'Epic', 'Libre Wolf', 'Opera', 'Firefox', 'Safari', 'Tor', 'Vivaldi', 'Yandex', 'Waterfox'])]
mega_df.head()

In [ ]:
# remove rows with result None

mega_df = mega_df[mega_df['result'].notna()]
mega_df.head()

In [ ]:
mega_df['test'].str.startswith('~')

In [50]:
# remove rows where test starts with ~, these differentiate tests with the same name

mega_df = mega_df[~mega_df['test'].str.startswith('~')]

## Replace values

In [ ]:
mega_df["result"].value_counts()

In [ ]:
# replace ✘, x with 0, and ✓ with 1

mega_df["result"] = mega_df["result"].replace("✘", 0)
mega_df["result"] = mega_df["result"].replace("x", 0)
mega_df["result"] = mega_df["result"].replace("✓", 1)
mega_df["result"].value_counts()

## Group by Category

### Reporting and Analytics

In [ ]:
reporting_and_analytics = [
    "Google Analytics",
    "Google Tag Manager",
    "Chartbeat",
    "New Relic",
    "Quantcast",
    "Scorecard Research Beacon"
]

reporting_df = mega_df[mega_df["test"].isin(reporting_and_analytics)]
reporting_df.head()

In [ ]:
reporting_df["result"].value_counts()

In [55]:
# remove empty rows

reporting_df = reporting_df[reporting_df["result"] != ""]

In [ ]:
# get average by browser and os for each test

reporting_df[['browser', 'os', 'result']].groupby(['browser', 'os']).mean()

### Behavioral Profiling

In [ ]:
behavioral_profiling = [
    "cookie (HTTP)",
    "cookie (JS)",
    "localStorage",
    "indexedDB",
    "ServiceWorker",
    "fetch cache",
    "document.referrer",
    "sessionStorage",
    "window.name",
    "Permissions API",
    "Geolocation API",
    "Geolocation",
    "Local storage",
    "Client hints",
    "DNS Leak Test",
    "IPv6 Leak Test",
    "Leaks of extension IDs via CSP",
    "Request blocking",
    "Storage partitioning test",

]

# set max rows to 500 in pandas
pd.set_option('display.max_rows', 500)

behav_df = mega_df[mega_df["test"].isin(behavioral_profiling)]


behav_df[['test','result']].value_counts()

In [ ]:
# if / in result, evaluate ratio of x/y


def compute_ratio(x):
    if type(x) != str:
        return x
    if x != "N/A" and "/" in x and 'pass' not in x:
        x = x.split("/")
        x = int(x[0]) / int(x[1])
    return x

behav_df["result"] = behav_df["result"].apply(compute_ratio)
behav_df["result"].value_counts()


# N/A fail for Epic, pass for Waterfox

replace = {
    "7 missing": 7/38,
    "6/21 pass": 6/21,
    "6/37 missing; 1 empty": 7/38,
    "1/23 blocked": 1/23,
    "1/21 blocked": 1/21,
    "2/21 blocked": 2/21,
    "1/22 blocked": 1/21,
    "not functional": 0,
    '-': 1
}

for key in replace:
    behav_df["result"] = behav_df["result"].replace(key, replace[key])

# if empty, delete row

behav_df = behav_df[behav_df["result"] != ""]

# if "N/A" and browser is Epic, fail, else pass

def compute_epic(x):
    if x["browser"] == "Epic" and x["result"] == "N/A":
        return 0
    elif x["browser"] == "Waterfox" and x["result"] == "N/A":
        return 1
    return x["result"]

# apply compute epic
behav_df["result"] = behav_df.apply(compute_epic, axis=1)

behav_df['result'].value_counts()

In [ ]:
# create new df with tests in HTML5 Features Detection CSS Features Detection Graphics Features Detection Storage Features Detection ECMAScript Features Detection Network Features Detection Miscellaneous

features_df = mega_df[mega_df["test"].isin(["HTML5 Features Detection", "CSS Features Detection", "Graphics Features Detection", "Storage Features Detection", "ECMAScript Features Detection", "Network Features Detection", "Miscellaneous"])]
features_df.head()

# if only 1 "/" in result, evaluate ratio of x/y

def compute_ratio(x):
    if type(x) != str:
        return x
    if x.count("/") == 1:
        x = x.split("/")
        x = int(x[0]) / int(x[1])
    return x

# apply compute ratio
features_df["result"] = features_df["result"].apply(compute_ratio)

# if test is HTML5 Features Detection, then split on "/" and calculate ((w) + (.66*y) + (.33*z))/90 for w/x/y/z

def compute_html5(x):
    if x["test"] == "HTML5 Features Detection":
        if x["result"] == "":
            return x
        y = x["result"].split("/")
        x["result"] = (int(y[0]) + (.66 * int(y[1])) + (.33 * int(y[2]))) / 90
    return x

# apply compute html5
features_df = features_df.apply(compute_html5, axis=1)
# drop rows with result ""
features_df = features_df[features_df["result"] != ""]

#convert result to float
features_df["result"] = features_df["result"].astype(float)

# now average test scores by browser and os

rez_df = features_df[['browser', 'os', 'result']].groupby(['browser', 'os']).mean()

# add rez_df as a test called features to behav_df

rez_df_reset = rez_df.reset_index().rename(columns={"result": "result"}).assign(test="features")
behav_df = pd.concat([behav_df, rez_df_reset], ignore_index=True)

In [61]:
# convert behav result to float

behav_df["result"] = behav_df["result"].astype(float)

In [ ]:
behav_df[behav_df["result"] == "-"]

In [ ]:
behav_df[['browser', 'os', 'result']].groupby(['browser', 'os']).mean()

cookie (HTTP) cookie (JS) indexedDB localStorage ServiceWorker

### Targeted Advertisements

In [ ]:
targeted_ads = [
    "Adobe",
    "Adobe Audience Manager",
    "Amazon adsystem",
    "AppNexus",
    "Bing Ads",
    "Criteo",
    "DoubleClick (Google)",
    "Facebook tracking",
    "Google (third-party ad pixel)",
    "Google Tag Manager",
    "Index Exchange",
    "Taboola",
    "Twitter pixel",
    "Yandex Ads",
    "__hsfp",
    "__hssc",
    "__hstc",
    "__s",
    "_hsenc",
    "_openstat",
    "dclid",
    "fbclid",
    "gclid",
    "hsCtaTracking",
    "mc_eid",
    "mkt_tok",
    "ml_subscriber",
    "ml_subscriber_hash",
    "msclkid",
    "oly_anon_id",
    "oly_enc_id",
    "rb_clickid",
    "s_cid",
    "vero_conc",
    "wickedid",
    "yclid",
    "Facebook click to load",
    "Youtube click to load",
    "Query parameters"
]

ads_df = mega_df[mega_df["test"].isin(targeted_ads)]

ads_df['result'].value_counts()

In [65]:
# remove empty rows

ads_df = ads_df[ads_df["result"] != ""]

In [ ]:
# replace "1/4 pass" with 0.25

ads_df["result"] = ads_df["result"].replace("1/4 pass", 0.25)
ads_df["result"].value_counts()

In [ ]:
ads_df[['browser', 'os', 'result']].groupby(['browser', 'os']).mean()

### Fingerprinting

In [ ]:
fingerprinting = [
    "Web Browser",
    "Canvas Support",
    "Canvas Fingerprinting",
    "Font Enumeration",
    "Geolocation API",
    "Unicode Glyphs",
    "TCP/IP Fingerpinting",
    "TLS FIngerprint",
    "HTTP/2 Fingerprint",
    "Image File Details",
    "Media query screen height",
    "Media query screen width",
    "outerHeight",
    "screen.height",
    "screen.width",
    "screenX",
    "screenY",
    "WebGL Fingerprint",
    "WebRTC Support Detection",
    "WebRTC IP Address Detection",
    "WebRTC Media Devices",
    "Harmful APIs",
    "WebGL Extensions",
    "Screen Object",
    "Date/Time",
    "Internationalization API",
    "Navigator Object",
    "Navigator.userAgentdata (Client Hints)",
    "navigator.plugins",
    "Batery Status API",
    "Network Information API",
    "HTML5 Features",
    "CSS Features"
]

fp_df = mega_df[mega_df["test"].isin(fingerprinting)]

fp_df[['test', 'result']].value_counts()

In [ ]:
# invert pass/fail WebRTC Support Detection

for i in fp_df.iterrows():
    x = fp_df.loc[i[0]]
    if x["test"] == "WebRTC Support Detection":
        if x["result"] == 1:
            x["result"] = 0
        elif x["result"] == 0:
            x["result"] = 1
        print(f"flipped {x['result']} for browser {x['browser']}")

# remove empty results
fp_df = fp_df[fp_df["result"] != ""]
fp_df["result"] = fp_df["result"].replace("1/3 passed", 1/3)

# apply compute
fp_df["result"].value_counts()

In [ ]:
# print test where result is N/A
fp_df[fp_df["result"] == "N/A"]

In [ ]:
# if result ends in unavailable, take the first word and compute the ratio, it is of form "x/y unavailable"

def compute_ratio(x):
    if type(x) != str:
        return x
    if "/" in x:
        x, y = x.split("/")[0], x.split("/")[1]
        return eval(x)/eval(y)
    return x
# replace not supported with 1, - with 1 , and "N/A" with 1

fp_df["result"] = fp_df["result"].replace("Not Supported", 1)
fp_df["result"] = fp_df["result"].replace("-", 1)
fp_df["result"] = fp_df["result"].replace("N/A", 1)

fp_df["result"] = fp_df["result"].apply(compute_ratio)
fp_df["result"].value_counts()

In [72]:
fp_protections = {
    "Brave": 1,
    "Chrome": 0,
    "Chromium": 0,
    "DuckDuckGo": 1,
    "Edge": 0,
    "Epic": 0.5,
    "Firefox": 1,
    "Libre Wolf": 0.5,
    "Opera": 0,
    "Safari": 1,
    "Tor": 1,
    "Vivaldi": 0.5,
    "Yandex": 0,
    "Waterfox": 0
}

In [ ]:
# take mean of fp_df like this fp_df[['browser', 'os', 'result']].groupby(['browser', 'os']).mean() but use .75 times fp_protections plus .25 times mean of fp_protections is 1
# .5 times mean of fp_protections is .5, and 0 times mean of fp_protections is 0.5
# and 0.25 times fp_protections plus .75 times mean of fp_protections is 0

# make sure all results are floats
fp_df["result"] = fp_df["result"].astype(float)
fp_means = fp_df[['browser', 'os', 'result']].groupby(['browser', 'os']).mean()

fp_results = pd.DataFrame(columns=["browser", "os", "result"])

for browser, score in fp_protections.items():
    if score == 1:
        final_score = (0.75 * score) + (0.25 * fp_means.loc[browser])
        for os in ["win", "mac"]:
            if os == "win" and browser == "Safari":
                continue
            fp_results = pd.concat([fp_results, pd.DataFrame([{
                "browser": browser,
                "os": os,
                "result": final_score["result"][os]
            }])], ignore_index=True)
    elif score == 0.5:
        final_score = (0.5 * score) + (0.5 * fp_means.loc[browser])
        for os in ["win", "mac"]:
            if os == "win" and browser == "Safari":
                continue
            fp_results = pd.concat([fp_results, pd.DataFrame([{
                "browser": browser,
                "os": os,
                "result": final_score["result"][os]
            }])], ignore_index=True)
    else:
        final_score = (0.25 * score) + (0.75 * fp_means.loc[browser])
        for os in ["win", "mac"]:
            if os == "win" and browser == "Safari":
                continue
            fp_results = pd.concat([fp_results, pd.DataFrame([{
                "browser": browser,
                "os": os,
                "result": final_score["result"][os]
            }])], ignore_index=True)

fp_results

In [ ]:
# concatenate fp_df, ads_df, reporting_df, behav_df

tests_final_df = pd.concat([fp_df, ads_df, reporting_df, behav_df], ignore_index=True)

tests_final_df


In [75]:
# behav using values 0.78	0.79	0.58	0.56	0.58	0.60	0.44	0.88	0.42	0.56	0.65	0.40	0.78	0.83	0.79	0.83	0.39	0.41	0.84	0.84	0.83	0.40	0.41	0.39	0.40	0.78	0.83
behav = [0.78,	0.79,	0.58,	0.56,	0.58,	0.60,	0.44,	0.88,	0.42,	0.56,	0.65,	0.40,	0.78,	0.83,	0.79,	0.83,	0.39,	0.41,	0.84,	0.84,	0.83,	0.40,	0.41,	0.39,	0.40,	0.78,	0.83]

fp = [0.85,	0.86,	0.14,	0.12,	0.16,	0.12,	0.81,	0.83,	0.16,	0.13,	0.32,	0.35,	0.82,	0.82,	0.48,	0.51,	0.16,	0.15,	0.82,	0.86,	0.90,	0.36,	0.33,	0.16,	0.13,	0.20,	0.27]
ads = [0.93,	0.93,	0.01,	0.01,	0.01,	0.01,	0.37,	0.49,	0.01,	0.01,	0.03,	0.01,	0.01,	0.01,	1.00,	1.00,	0.01,	0.01,	0.01,	0.03,	0.47,	0.01,	0.01,	0.01,	0.01,	0.01,	0.01]
report = [0.93,	0.93,	0.01,	0.01,	0.01,	0.01,	0.37,	0.49,	0.01,	0.01,	0.03,	0.01,	0.01,	0.01,	1.00,	1.00,	0.01,	0.01,	0.01,	0.03,	0.47,	0.01,	0.01,	0.01,	0.01,	0.01,	0.01]

In [ ]:
import numpy as np
import pandas as pd

# min max scale each list
def min_max_scale(x):
    x = np.array(x)
    return (x - min(x)) / (max(x) - min(x))

behav = min_max_scale(behav)
fp = min_max_scale(fp)
ads = min_max_scale(ads)
report = min_max_scale(report)

# take mean by index of four list, so average of first element of each list, ...

final = []
for i in range(len(behav)):
    final.append((behav[i] + fp[i] + ads[i] + report[i]) / 4)

final

#associate with each browser and os by index in the order 'Brave' (win, macos), 'Chrome' (win,macos), 'Chromium', 'DuckDuckGo', 'Edge', 'Epic', 'Libre Wolf', 'Opera', 'Firefox', 'Safari', 'Tor', 'Vivaldi', 'Yandex', 'Waterfox' alternating win and macos, with win first

final_df = pd.DataFrame(final, columns=["result"])

final_df["browser"] = ["Brave", "Brave", "Chrome", "Chrome", "Chromium", "Chromium", "DuckDuckGo", "DuckDuckGo", "Edge", "Edge", "Epic", "Epic","Firefox", "Firefox", "Libre Wolf", "Libre Wolf", "Opera", "Opera",   "Safari", "Tor", "Tor", "Vivaldi", "Vivaldi", "Yandex", "Yandex", "Waterfox", "Waterfox"]
# os column, W11	MacOS	W11	MacOS	W11	MacOS	W11	MacOS	W11	MacOS	W11	MacOS	W11	MacOS	W11	MacOS	W11	MacOS	W11	MacOS	W11	MacOS	W11	MacOS	W11	MacOS	W11	MacOS
final_df["os"] = ["W11",	"MacOS", "W11",	"MacOS", "W11",	"MacOS",	"W11",	"MacOS",	"W11",	"MacOS",	"W11",	"MacOS",	"W11",	"MacOS",	"W11",	"MacOS",	"W11",	"MacOS",	"MacOS",	"W11",	"MacOS",	"W11",	"MacOS",	"W11",	"MacOS",	"W11",	"MacOS"]

final_df
    

In [ ]:
!pip install jenkspy

In [ ]:
import jenkspy

# get breaks in results and categories the browser + os combos based on the cutoffs

breaks = jenkspy.jenks_breaks(final_df["result"], n_classes=3)
print(breaks)

def categorize(x):
    if x <= breaks[1]:
        return "low"
    elif x <= breaks[2]:
        return "medium"
    else:
        return "high"

final_df["category"] = final_df["result"].apply(categorize)
final_df

In [ ]:
!pip install matplotlib

In [82]:
avg_df = final_df.groupby('browser')['result'].mean().reset_index()

In [ ]:
# generate number line with intervals highlighted red, yellow, green
import matplotlib.pyplot as plt

cutoffs = [0.22462061747776033, 0.4768723340151911]

fig, ax = plt.subplots(figsize=(10, 4))
colors = ["red", "yellow", "green"]

# Highlight the intervals
for i in range(len(cutoffs) + 1):
    left = cutoffs[i - 1] if i != 0 else -0.05
    right = cutoffs[i] if i != len(cutoffs) else 1
    print(left, right)
    ax.fill_between([left, right], -0.5, 3, color=colors[i % len(colors)], alpha=0.5)

spacing = .2  # This can be adjusted for more or less spacing
y_coords = [-0.4 + i*(3/len(avg_df)) for i in range(len(avg_df))]
for y, (_, row) in zip(y_coords, avg_df.iterrows()):
    if pd.notna(row['result']):
        ax.text(row['result'], y, row['browser'], ha='center', va='center')

# Customize axis
ax.set_yticks([])
ax.set_xlim(avg_df['result'].min() - 0.05, avg_df['result'].max() + 0.05)  # Small buffer around limits
ax.set_ylim(-0.5, 3)
ax.set_xlabel('Composite Score')


plt.tight_layout()
plt.show()


In [ ]:
# order avg_df by result desc
avg_df = avg_df.sort_values(by=['result'], ascending=False)
avg_df

In [ ]:
# concatenate fp_df, ads_df, reporting_df, behav_df

tests_final_df = pd.concat([fp_df, ads_df, reporting_df, behav_df], ignore_index=True)

tests_final_df
